In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# X_train = X_train.reshape(X_train.shape[0],28,28,1)
# X_test = X_test.reshape(X_test.shape[0],28,28,1)

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax'),
])

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/3
938/938 [==============================] - 12s 8ms/step - loss: 0.1778 - accuracy: 0.9486 - val_loss: 0.0764 - val_accuracy: 0.9769
Epoch 2/3
938/938 [==============================] - 7s 7ms/step - loss: 0.0598 - accuracy: 0.9821 - val_loss: 0.0534 - val_accuracy: 0.9819
Epoch 3/3
938/938 [==============================] - 7s 7ms/step - loss: 0.0393 - accuracy: 0.9879 - val_loss: 0.0483 - val_accuracy: 0.9841


In [9]:
feature_extractor = Model(inputs=model.input, outputs=model.layers[-2].output)

In [10]:
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

313/313 [==============================] - 1s 1ms/step


In [11]:
ada = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=50)

In [12]:
from sklearn.model_selection import KFold, cross_val_score
kfold = KFold(n_splits=5)
cv_scores = cross_val_score(ada, X_train_features, y_train, cv=kfold, scoring='accuracy')

In [13]:
print(f"Cross-validation scores: {cv_scores}")
print(f"Average cross-validation score: {np.mean(cv_scores)}")

Cross-validation scores: [0.95358333 0.82441667 0.95291667 0.89058333 0.89633333]
Average cross-validation score: 0.9035666666666666


In [14]:
ada.fit(X_train_features, y_train)
y_pred = ada.predict(X_test_features)

In [15]:
print("Boosted Model Accuracy:", accuracy_score(y_test, y_pred))

Boosted Model Accuracy: 0.9508
